[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=PracticalRunMarketmaker.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Practical: Run Marketmaker

This notebook shows how to load a custom `OrderChain` install it in `MarketMaker`, and then run that `MarketMaker` on a market.

This Practical is a bigger task than the normal examples here, so it is split into the following sections:
1. Imports and Setup
2. Build the `Context`
3. Build the `OrderChain`, including instantiating all the `Elements`
4. Build the `OrderReconciler`
5. Build the `Oracle`
6. Build the `ModelState` - this function will be called every 'pulse' to fetch the latest data
7. Build the `MarketMaker`
8. Run the `MarketMaker`

## 1. Imports and Setup

This is just some basic housekeeping. (These are the only imports used in this example.)

The line:
```
    logging.getLogger().setLevel(logging.DEBUG)
```
sets up logging and allows messages of DEBUG or higher to be shown on the page when running code. This can log a lot of information to the page, slowing things down, so `logging.INFO` is a useful compromise. If running the code on this page for longer than a few minutes, you might be better limiting it to `logging.ERROR` so that only errors or critical problems are shown.

In [ ]:
import logging
import mango
import mango.marketmaking
import time

from datetime import timedelta
from decimal import Decimal

from mango.marketmaking.orderchain.chain import Chain
from mango.marketmaking.orderchain.biasquoteonpositionelement import BiasQuoteOnPositionElement
from mango.marketmaking.orderchain.ratioselement import RatiosElement
from mango.marketmaking.orderchain.roundtolotsizeelement import RoundToLotSizeElement

# Set logging so messages show up.
logging.getLogger().setLevel(logging.INFO)

## 2. Build the `Context`

The `Context` object provides access to the Solana RPC nodes, as well as knowing the current Mango `Group` and hosting some static data like token mint addresses and symbols.

Typically only one `Context` is used per program, and it doesn't usually need to be re-created or refreshed. Multiple `Context`s can be created too, if required, and they shouldn't interfere with each other.

Here we create a 'devnet' `Context` using all the default options. A 'mainnet' `Context` using all the default options could be created using:
```
    return mango.ContextBuilder.build()
```
instead.

More typically, some options for communicating with the Solana RPC node may be preferred:
```
    return mango.ContextBuilder.build(cluster_name="devnet", cluster_url="https://api.devnet.solana.com", commitment="confirmed", skip_preflight=True)
```

### References

* [Show Context example](ShowContext.ipynb)

In [ ]:
def build_context():
    return mango.ContextBuilder.build(cluster_name="devnet")

## 3. Build the `OrderChain`, including instantiating the `Elements` you want to use

This merits a bit of explanation. There's a more detailed explanation of what an `OrderChain` is, what `Elements` are available (and what configuration options they provide), in the [documentation](https://github.com/blockworks-foundation/mango-explorer/blob/main/docs/MarketmakingOrderChain.md).

An `OrderChain` is what builds the `Order`s you want the marketmaker to place - the 'desired' `Order`s. Typically the first `Element` builds a list of `Order`s, and subsequent (optional) `Element`s add, remove or change those orders. After all `Element`s have had the chance to operate on the `Order`s, the list of these 'desired' `Order`s is ready for the next stage.

For example, the first `Element` could be a `RatiosElement` that builds a BUY and SELL order at a spread of 0.5% and a position size of 5% of inventory. Then the next `Element` could be a `BiasQuoteOnPositionElement` to adjust the price of those `Order`s based on the current 'position':
- if the position is net-short, it would adjust the price down to tend to buy more, or
- if the position is net-long, it would adjust the price up to tend to sell more

The final element could then round the price and quantity values to the proper 'lot sizes' for the market. Having this as the last stage means it only needs to be done once, and it makes it easier to compare `Order`s when it comes to 'reconciliation' (see the next step).

Keeping the functionality in separate `Element`s makes the system more configurable and much easier to test.

This code builds that example `OrderChain`, with a `RatiosElement`, a `BiasQuoteOnPositionElement`, and a `RoundToLotSizeElement`.

### References

* [OrderChain documentation](https://github.com/blockworks-foundation/mango-explorer/blob/main/docs/MarketmakingOrderChain.md)

In [ ]:
def build_order_chain():
    order_type = mango.OrderType.POST_ONLY
    spread = Decimal("0.005")  # 0.5%
    position_size = Decimal("0.05")  # 5%
    bias = [Decimal("0.0001")]

    ratios_element = RatiosElement(order_type, None, 20, [spread], [position_size], False)
    bias_element = BiasQuoteOnPositionElement(bias)
    round_to_lot_size_element = RoundToLotSizeElement()

    chain = Chain([ratios_element, bias_element, round_to_lot_size_element])

    return chain

## 4. Build the `OrderReconciler`

After the chain has build the list of 'desired' `Order`s, the marketmaker checks the orderbook to see what `Order`s it currently has open.

The list of desired `Order`s and the list of open `Order`s are compared, to return:
* A list of existing `Order`s to cancel
* A list of existing `Order`s to keep
* A list of fresh `Order`s to place

This 'reconciliation' of `Order`s is performed by an `OrderReconciler`, and depending on how strict it is `Order`s may be replaced on nearly every 'pulse' or may be left on the orderbook for many 'pulses'.

We'll use a `ToleranceOrderReconciler`, and we'll set it so that orders are kept on the orderbook (not cancelled, not replaced) if their prices and quantities are within 0.1% of a 'desired' `Order` from the `OrderChain`.

In [ ]:
def build_order_reconciler():
    return mango.marketmaking.ToleranceOrderReconciler(Decimal("0.001"), Decimal("0.001"), timedelta(seconds=5))


## 5. Build the `Oracle`

The `Oracle` is the source of price information for our marketmaker. There are a number of possible oracle sources we could use:
* Pyth
* FTX
* Serum

For this example, we'll use Pyth.

We construct the `Oracle` from the `OracleProvider`, and return it. The `Oracle` provides a `fetch_price()` method which will always fetch the latest price, so the `Oracle` itself can be safely reused or cached.

### References

* [Oracle documentation](https://github.com/blockworks-foundation/mango-explorer/blob/main/docs/MarketmakingOracle.md)

In [ ]:
def build_oracle(context, market):
    oracle_source_name = "pyth"
    oracle_provider = mango.create_oracle_provider(context, oracle_source_name)
    oracle = oracle_provider.oracle_for_market(context, market)
    if oracle is None:
        raise Exception(f"Could not find oracle for market {market.symbol} from provider {oracle_source_name}.")
    return oracle


## 6. Build the `ModelState`

Unlike the first two functions - `build_context()` and `build_order_chain()` - this function will be called __frequently__. It will be called every 'pulse' to fetch the latest data, so it's important that it's efficient in fetching the data.

The `ModelState` can be used two ways:
1. It can be created fresh for each 'pulse', or
2. It can take advantage of `Watcher`s, which can automatically update themselves (for example via websocket updates)

This code will assume the first approach and will create a fresh `ModelState` each time `build_model_state()` is called. This means many of our objects need to be wrapped in a `ManualUpdateWatcher` class when being passed to the `ModelState` constructor, but all that does here is provide a wrapper with a `latest` property that points to the object we just provided.

In [ ]:
def build_model_state(context, order_owner, market, oracle, group_address, cache_address, account_address, all_open_orders_addresses):
    # Build a list of addresses and use `load_multiple()` to fetch them all in one go.
    addresses = [
        group_address,
        cache_address,
        account_address,
        market.bids_address,
        market.asks_address,
        market.event_queue_address,
        *all_open_orders_addresses,
    ]
    account_infos = mango.AccountInfo.load_multiple(context, addresses)
    group = mango.Group.parse_with_context(context, account_infos[0])
    cache = mango.Cache.parse(account_infos[1])
    account = mango.Account.parse(account_infos[2], group, cache)

    group_slot = group.slot_by_perp_market_address(market.address)
    perp_account = account.perp_accounts[group_slot.index]
    if perp_account is None:
        raise Exception(f"Could not find perp account at index {group_slot.index} of account {account.address}.")
    placed_orders_container = perp_account.open_orders

    base_lots = perp_account.base_position
    base_value = market.lot_size_converter.base_size_lots_to_number(base_lots)
    base_token_value = mango.InstrumentValue(market.base, base_value)
    quote_token_value = account.shared_quote.net_value

    event_queue = mango.PerpEventQueue.parse(account_infos[5], market.lot_size_converter)

    spot_open_orders_account_infos_by_address = {
        str(account_info.address): account_info
        for account_info in account_infos[6:]
    }

    all_open_orders = {}
    for basket_token in account.slots:
        if (
            basket_token.spot_open_orders is not None
            and str(basket_token.spot_open_orders)
            in spot_open_orders_account_infos_by_address
        ):
            account_info: mango.AccountInfo = (
                spot_open_orders_account_infos_by_address[
                    str(basket_token.spot_open_orders)
                ]
            )
            open_orders: mango.OpenOrders = mango.OpenOrders.parse(
                account_info,
                basket_token.base_instrument,
                account.shared_quote_token,
            )
            all_open_orders[str(basket_token.spot_open_orders)] = open_orders

    frame = account.to_dataframe(group, all_open_orders, cache)
    available_collateral = account.init_health(frame)
    inventory = mango.Inventory(mango.InventorySource.ACCOUNT,
                                perp_account.mngo_accrued,
                                available_collateral,
                                base_token_value,
                                quote_token_value)

    orderbook = market.parse_account_infos_to_orderbook(account_infos[3], account_infos[4])

    price = oracle.fetch_price(context)

    return mango.ModelState(order_owner,
                            market,
                            mango.ManualUpdateWatcher(group),
                            mango.ManualUpdateWatcher(account),
                            mango.ManualUpdateWatcher(price),
                            mango.ManualUpdateWatcher(placed_orders_container),
                            mango.ManualUpdateWatcher(inventory),
                            mango.ManualUpdateWatcher(orderbook),
                            mango.ManualUpdateWatcher(event_queue))

## 7. Build the `MarketMaker`

Compared to all that build-up above, constructing the `MarketMaker` is now relatively straightforward!

In [ ]:
def build_marketmaker(context, wallet, account, market):
    chain = build_order_chain()
    order_reconciler = build_order_reconciler()
    instruction_builder = mango.instruction_builder(context, wallet, account, market.symbol, False)
    market_maker = mango.marketmaking.MarketMaker(wallet, market, instruction_builder, chain, order_reconciler, Decimal(0))
    return market_maker


## 8. Run the `MarketMaker`

That's a lot of setup code above, but it also shows you which bits you can change and introduce your own 'smarts'.

Now we just need to call those functions and start the marketmaker! We do this in a simple loop, with a `sleep()` call after it. You can stop the loop by pressing the Stop (⏹️) button in the toolbar.

### Caveat

Please bear in mind that the code below uses a shared Solana `Keypair` and a shared Mango `Account`. If you're going to run this example more than once or twice, it might be better for you to set up a fresh devnet `Keypair` and `Account` for you to use - it's easy and free, and it means other people running this example won't accidentally cancel your orders or cause confusion.

In [ ]:
# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = mango.Wallet(bytes([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146]))

# Specify the market we're going to use
market_symbol = "SOL-PERP"

# Configure how long to pause between pulses
pause_seconds = 30

# Create a 'devnet' Context
context = build_context()

# Load the wallet's account
group = mango.Group.load(context)
accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
account = accounts[0]

# Load the market
market = mango.market(context, market_symbol)

oracle = build_oracle(context, market)
marketmaker = build_marketmaker(context, wallet, account, market)

stop_requested = False
while not stop_requested:
    try:
        model_state = build_model_state(context, account.address, market, oracle, group.address, group.cache, account.address, account.spot_open_orders)
        marketmaker.pulse(context, model_state)

        # Wait and hope for fills.
        print(f"Pausing for {pause_seconds} seconds.\n")
        time.sleep(pause_seconds)
    except KeyboardInterrupt:
        stop_requested = True
    except Exception as exception:
        print(f"Continuing after problem running market-making iteration: {exception}")

market_operations = mango.operations(context, wallet, account, market, dry_run=False)
for order in market_operations.load_my_orders():
    market_operations.cancel_order(order, ok_if_missing=True)

context.dispose()
print("Example complete")
